# Get Embeddings

In [32]:
%load_ext autoreload
%autoreload 2

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
!pip install openai
!pip install load_dotenv
!pip install azure-identity
!pip install azure-mgmt-cognitiveservices


[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


## Load Data

In [104]:
import pandas as pd

df_orig = pd.read_csv("bbc-news-data.csv", delimiter='\t')
df = df_orig.copy()
df

,category,filename,title,content
0,business,001.txt,Ad sales boost Time Warner profit,Quarterly profits at US media giant TimeWarne...
1,business,002.txt,Dollar gains on Greenspan speech,The dollar has hit its highest level against ...
2,business,003.txt,Yukos unit buyer faces loan claim,The owners of embattled Russian oil giant Yuk...
3,business,004.txt,High fuel prices hit BA's profits,British Airways has blamed high fuel prices f...
4,business,005.txt,Pernod takeover talk lifts Domecq,Shares in UK drinks and food firm Allied Dome...
...,...,...,...,...
2241,tech,397.txt,BT program to beat dialler scams,BT is introducing two initiatives to help bea...
2242,tech,398.txt,Spam e-mails tempt net shoppers,Computer users across the world continue to i...
2243,tech,399.txt,Be careful how you code,A new European directive could put software w...
2244,tech,400.txt,US cyber security chief resigns,The man making sure US computer networks are ...


## Deploy a model

In [91]:
from azure.identity import DefaultAzureCredential
from azure.mgmt.cognitiveservices import CognitiveServicesManagementClient

os.environ['AZURE_CLIENT_ID']=""
os.environ['AZURE_TENANT_ID']=""
os.environ['AZURE_CLIENT_SECRET']=""

# list models deployed with embeddings capability
deployment_id = None
client = CognitiveServicesManagementClient(
        credential=DefaultAzureCredential(),
        subscription_id="your-cluster-subs-id",
    )

response = client.deployments.begin_create_or_update(
        resource_group_name="cloud-shell-storage-eastus",
        account_name="tsisodia-openai",
        deployment_name="tsisodia-gpt-1",
        deployment={
            "properties": {"model": {"format": "OpenAI", "name": "text-embedding-ada-002", "version": "2"}},
            "sku": {"capacity": 2, "name": "Standard"},
        },
    )

response = client.deployments.list(
        resource_group_name="cloud-shell-storage-eastus",
        account_name="tsisodia-openai",
    )
for item in response:
    print(item)

{'additional_properties': {}, 'id': '/subscriptions/99d7836d-942d-4759-80b6-2ed3b80b49e9/resourceGroups/cloud-shell-storage-eastus/providers/Microsoft.CognitiveServices/accounts/tsisodia-openai/deployments/tsisodia-gpt4o', 'name': 'tsisodia-gpt4o', 'type': 'Microsoft.CognitiveServices/accounts/deployments', 'sku': <azure.mgmt.cognitiveservices.models._models_py3.Sku object at 0x7f089d02e4f0>, 'system_data': <azure.mgmt.cognitiveservices.models._models_py3.SystemData object at 0x7f089d02e5e0>, 'etag': '"f0fc90f3-1a6b-4d58-924c-041e474b364f"', 'properties': <azure.mgmt.cognitiveservices.models._models_py3.DeploymentProperties object at 0x7f089d02e640>}
{'additional_properties': {}, 'id': '/subscriptions/99d7836d-942d-4759-80b6-2ed3b80b49e9/resourceGroups/cloud-shell-storage-eastus/providers/Microsoft.CognitiveServices/accounts/tsisodia-openai/deployments/tsisodia-gpt', 'name': 'tsisodia-gpt', 'type': 'Microsoft.CognitiveServices/accounts/deployments', 'sku': <azure.mgmt.cognitiveservices

In [105]:
# Use the latest text embedding model
import os
import openai
from dotenv import load_dotenv
from openai import AzureOpenAI

client = AzureOpenAI(
  api_key = "openAI_key",
  api_version = "2023-05-15",
  azure_endpoint = "https://<name>.openai.azure.com/"  
)
embedding_response = client.embeddings.create(
    input="Hello",
    model='tsisodia-gpt'
    )

embedding = embedding_response.data[0].embedding

print("Embedding the text:", embedding)

Embedding the text: [-0.021819502115249634, -0.007147768046706915, -0.028617016971111298, -0.02436208352446556, -0.023570770397782326, 0.029084021225571632, -0.01248589251190424, -0.00280202878639102, -0.00822447333484888, -0.0051078652031719685, 0.029317524284124374, -0.0032560613472014666, -0.01567060686647892, -0.0025928495451807976, 0.012174556031823158, -0.0009461712324991822, 0.0385797880589962, 0.005762969143688679, 0.019004501402378082, -0.013672863133251667, -0.01980878785252571, 0.010098978877067566, 0.0050689480267465115, 0.008808230049908161, -0.008127180859446526, -0.005315423011779785, 0.002570148091763258, -0.012122666463255882, 0.0033857848029583693, -0.015722494572401047, 0.0038106294814497232, -0.016176527366042137, -0.017849961295723915, -0.012926952913403511, 0.004057104233652353, -0.016306251287460327, -0.0009186049574054778, -0.009768184274435043, 0.021508166566491127, -0.008406086824834347, 0.012965870089828968, -0.006460233125835657, 0.003580370219424367, -0.006

In [106]:
df['embedding'] = ''
for i in range(len(df.head(20))):
    print(f"Length of content at index {i}: {len(df['content'][i])}")
for i in range(len(df.head(20))):
    try:
        embedding_response = client.embeddings.create(input=df['content'][i], model='tsisodia-gpt')
        df['embedding'][i] = embedding_response.data[0].embedding  
    except Exception as err:
        i
        print(f"Unexpected {err=}, {type(err)=}")     

Length of content at index 0: 2525
Length of content at index 1: 2219
Length of content at index 2: 1517
Length of content at index 3: 2367
Length of content at index 4: 1535
Length of content at index 5: 1154
Length of content at index 6: 1636
Length of content at index 7: 1889
Length of content at index 8: 1460
Length of content at index 9: 1117
Length of content at index 10: 1548
Length of content at index 11: 1047
Length of content at index 12: 1219
Length of content at index 13: 1111
Length of content at index 14: 1380
Length of content at index 15: 2097
Length of content at index 16: 1775
Length of content at index 17: 1939
Length of content at index 18: 1910
Length of content at index 19: 1212


## Save embeddings
Following notebooks will require embeddings generated in this notebook. 

In [108]:
# Save embedding
df.to_csv("bbc-news-data-embedding.csv", sep='\t', index=False)